In [4]:
#%pip install transformers --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
#%pip install torch --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [19]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [20]:
def load_and_prepare(path):
    df = pd.read_csv(path, on_bad_lines='skip')
    df['label'] = df['Оценка'].apply(lambda x: 0 if x < 3 else (1 if x == 3 else 2))
    df = df.dropna(subset=['Отзыв','Оценка'])
    return df

In [21]:
df = load_and_prepare('yandex_reviews_main_last_right_version.csv')

In [22]:
df

,Название точки,Ссылка,Отзыв,Оценка,Тип организации,label
0,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Этот медицинский центр, лучшее, что мы видели ...",5.0,Детская поликлиника,2
1,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Очень приятные, вежливые и предусмотрительные ...",5.0,Детская поликлиника,2
2,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Редкий, крутейший и очень ответственный специа...",5.0,Детская поликлиника,2
3,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,Хочу выразить свою благодарность Болдыреву Вла...,5.0,Детская поликлиника,2
4,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,Уже более пяти лет посещаем врача Владимира Ге...,5.0,Детская поликлиника,2
...,...,...,...,...,...,...
106011,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Советую посещать данное заведение!,5.0,Госпиталь,2
106012,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,"Отличные врачи, новое оборудование. Спасибо за...",5.0,Госпиталь,2
106013,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Скоро прийдёт новый начальник и надеюсь постав...,3.0,Госпиталь,1
106014,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Отличный госпиталь.,5.0,Госпиталь,2


In [ ]:
import re
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s.,!?]', '', text)
    return text.strip()

df['Отзыв'] = df['Отзыв'].apply(clean_text)

In [24]:
train_df, temp_df = train_test_split(
    df, test_size=0.3, stratify=df['label'], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.25, stratify=temp_df['label'], random_state=42
)


In [25]:
class_weights = compute_class_weight(
    'balanced', classes=np.unique(train_df['label']), y=train_df['label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [26]:
class_weights

tensor([2.0521, 6.8099, 0.4227])

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'ai-forever/ruRoberta-large'

tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer)
max_length = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [28]:
import warnings
warnings.filterwarnings("ignore")

class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.encodings = tokenizer(
            texts.tolist(),
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels.tolist(), dtype=torch.long)

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

In [29]:
train_ds = ReviewDataset(train_df['Отзыв'], train_df['label'], tokenizer, max_length)
val_ds   = ReviewDataset(val_df['Отзыв'], val_df['label'], tokenizer, max_length)
test_ds  = ReviewDataset(test_df['Отзыв'], test_df['label'], tokenizer, max_length)

In [30]:
def compute_loss_with_weights(self, model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
    loss = loss_fct(logits, labels)
    return (loss, outputs) if return_outputs else loss

type(model).compute_loss = compute_loss_with_weights

In [31]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds  = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [32]:
training_args = TrainingArguments(
    output_dir='./out',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    #logging_dir='./logs',
    #logging_steps=100,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    learning_rate=3e-5,
    save_total_limit=2
)

In [17]:
#%pip install -U accelerate --upgrade

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [34]:
trainer.train()

 22%|██▏       | 500/2316 [09:23<33:44,  1.11s/it]

{'loss': 0.2303, 'grad_norm': 1.9513680934906006, 'learning_rate': 2.3536269430051816e-05, 'epoch': 0.86}


 99%|█████████▉| 186/187 [01:10<00:00,  2.66it/s]
                                                  [A
100%|██████████| 187/187 [01:10<00:00,  3.25it/s]
                                                 

{'eval_loss': 0.20322498679161072, 'eval_accuracy': 0.9277381451951322, 'eval_f1': 0.9175607106585738, 'eval_precision': 0.9132316389501955, 'eval_recall': 0.9277381451951322, 'eval_runtime': 71.3061, 'eval_samples_per_second': 334.193, 'eval_steps_per_second': 2.622, 'epoch': 1.0}


 43%|████▎     | 1000/2316 [21:08<24:54,  1.14s/it]  

...

 99%|█████████▉| 186/187 [01:10<00:00,  2.72it/s]
                                                   A
100%|██████████| 187/187 [01:11<00:00,  3.32it/s]
                                                 

{'eval_loss': 0.2465813308954239, 'eval_accuracy': 0.9279060008392782, 'eval_f1': 0.92873419124401, 'eval_precision': 0.9296288895814235, 'eval_recall': 0.9279060008392782, 'eval_runtime': 71.4511, 'eval_samples_per_second': 333.515, 'eval_steps_per_second': 2.617, 'epoch': 3.99}


100%|██████████| 2316/2316 [53:02<00:00,  1.37s/it]

{'train_runtime': 3182.4306, 'train_samples_per_second': 93.185, 'train_steps_per_second': 0.728, 'train_loss': 0.1512236092786509, 'epoch': 3.99}


TrainOutput(global_step=2316, training_loss=0.1512236092786509, metrics={'train_runtime': 3182.4306, 'train_samples_per_second': 93.185, 'train_steps_per_second': 0.728, 'total_flos': 1.3799387304707328e+17, 'train_loss': 0.1512236092786509, 'epoch': 3.993960310612597})

In [35]:
metrics = trainer.evaluate(test_ds)
print("Test metrics:", metrics)

100%|██████████| 63/63 [00:22<00:00,  2.76it/s]

Test metrics: {'eval_loss': 0.24100321531295776, 'eval_accuracy': 0.9296324269889225, 'eval_f1': 0.9292274672361427, 'eval_precision': 0.9288498632454488, 'eval_recall': 0.9296324269889225, 'eval_runtime': 23.2275, 'eval_samples_per_second': 342.008, 'eval_steps_per_second': 2.712, 'epoch': 3.993960310612597}


In [36]:
from torch.utils.data import DataLoader, TensorDataset

In [37]:
texts = df['Отзыв'].tolist()
enc = tokenizer(texts, max_length=max_length, padding='max_length',
                truncation=True, return_tensors='pt')
dataset = TensorDataset(enc['input_ids'], enc['attention_mask'])

loader = DataLoader(dataset, batch_size=32)

model.eval()
all_probs = []
with torch.no_grad():
    for input_ids, attention_mask in loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        probs = torch.nn.functional.softmax(logits, dim=1).cpu()
        all_probs.append(probs)

probs = torch.cat(all_probs, dim=0).numpy()

In [38]:
import os


output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.json',
 './model_save/merges.txt',
 './model_save/added_tokens.json',
 './model_save/tokenizer.json')

In [7]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification


#output_dir = './out/checkpoint-2316/'
#model_name = 'DeepPavlov/xlm-roberta-large-en-ru'
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#model = AutoModelForSequenceClassification.from_pretrained(output_dir)
#tokenizer = AutoTokenizer.from_pretrained(output_dir)

#model.to(device)

#max_length = 256



In [42]:
import pandas as pd
import numpy as np


probs_df = pd.DataFrame(
    probs,
    columns=['p_neg', 'p_neu', 'p_pos']
)
probs_df.index = df.index

probs_df['company'] = df['Название точки']  # или df['Компания'] и т.п.

# нормированная разность
probs_df['norm_diff'] = (probs_df['p_pos'] - probs_df['p_neg']) / (
    probs_df['p_pos'] + probs_df['p_neg'] + 1e-12
)
# взвешенный индекс
probs_df['weighted_score'] = (
    0 * probs_df['p_neg'] +
    0.5 * probs_df['p_neu'] +
    1 * probs_df['p_pos']
)
# доля положительных среди ненейтральных
probs_df['pos_over_non_neu'] = probs_df['p_pos'] / (
    probs_df['p_pos'] + probs_df['p_neg'] + 1e-12
)

agg = probs_df.groupby('company').agg(
    avg_p_positive        = ('p_pos', 'mean'),
    avg_norm_diff         = ('norm_diff', 'mean'),
    avg_weighted_score    = ('weighted_score', 'mean'),
    avg_pos_over_non_neu  = ('pos_over_non_neu', 'mean'),
    n_reviews             = ('p_pos', 'size')  # на всякий случай
).reset_index()

In [43]:
agg

,company,avg_p_positive,avg_norm_diff,avg_weighted_score,avg_pos_over_non_neu,n_reviews
0,3D Medica,0.929921,0.921447,0.945570,0.960724,26
1,9 Лечебно-диагностический центр Минобороны России,0.897361,0.808058,0.907601,0.904029,210
2,9 Лечебно-диагностический центр Минобороны Рос...,0.880016,0.804810,0.897941,0.902405,599
3,ABC-медицина,0.825752,0.679008,0.841534,0.839504,715
4,Advanced Fertility Clinic,0.728107,0.489528,0.745294,0.744764,37
...,...,...,...,...,...,...
273,Центральный военный клинический госпиталь имен...,0.923272,0.873933,0.935231,0.936966,183
274,Эксимер,0.749775,0.547445,0.776407,0.773723,506
275,ЭммаКлиник,0.706448,0.442261,0.720504,0.721131,48
276,Эстетическая клиника Век Адалин,0.911737,0.831657,0.917396,0.915829,155


In [45]:
agg.to_csv('companies_ethics_indexes_ai-forever_ruRoberta-large.csv',index=False)